# Проект по SQL

Компания купила крупный сервис для чтения книг по подписке

## Цели исследования
- Проанализировать базу данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг.
- Cформулировать ценностное предложение для нового продукта

## Описание данных

Таблица `books` Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.



Таблица `authors` Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.


Таблица `publishers` Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;


Таблица ratings Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.


Таблица `reviews` Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

## Задания
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключите из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

# развернуть наименование
pd.set_option('display.max_colwidth', None)

In [3]:
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']
query_1 = "SELECT * FROM " 
query_2 = " LIMIT(1);"
for table in tables:
    print(table)
    query_1 +str(table) + query_2
    display(pd.io.sql.read_sql(query_1 +str(table) + query_2, con = engine))
    print()
    

books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93



authors


,author_id,author
0,1,A.S. Byatt



publishers


,publisher_id,publisher
0,1,Ace



ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4



reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.


## Посчитаем, сколько книг вышло после 1 января 2000 года

In [4]:
query = '''
SELECT COUNT(DISTINCT book_id) AS count_books
FROM books
WHERE publication_date > '2000-01-01';
'''
pd.io.sql.read_sql(query, con = engine)

,count_books
0,819


## Вывод
После 1 января 2000 года вышло 819 книг

## Для каждой книги посчитаем количество обзоров и среднюю оценку

In [5]:
query = '''
SELECT b.book_id,
       b.title,
       COUNT(DISTINCT review_id) AS count_review,
       AVG(rating) AS avg_rating
FROM books AS b
LEFT OUTER JOIN reviews AS rev ON b.book_id = rev.book_id
LEFT OUTER JOIN ratings AS rat ON b.book_id = rat.book_id
GROUP BY b.book_id
ORDER BY  AVG(rating) DESC;
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,count_review,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #1-12),2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Meditation in Everyday Life,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman's Soul,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twenty-first Century,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


## Вывод
- Максимальный средний рейтинг у книг **про медитации, про раскрытие женственности, про фантастику, про реальные истории успеха и у комиксов**.

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [6]:
query = '''
SELECT p.publisher,
       COUNT(DISTINCT b.book_id) AS count_books
FROM books AS b
JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY COUNT(DISTINCT b.book_id) DESC
LIMIT(1);
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count_books
0,Penguin Books,42


## Вывод
- Издательство, которое выпустило наибольшее число книг - **Penguin Books**

## Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [7]:
query = '''

WITH
q AS (
    SELECT b.book_id,
           b.author_id,
           COUNT(r.rating)
    FROM books AS b
    JOIN ratings AS r ON b.book_id = r.book_id
    GROUP BY b.book_id
    HAVING COUNT(r.rating) >= 50
    )
SELECT a.author,
       AVG(r.rating) AS avg_rating
FROM q 
JOIN authors AS a ON q.author_id = a.author_id
JOIN ratings AS r ON q.book_id = r.book_id    
GROUP BY a.author
ORDER BY AVG(r.rating) DESC
LIMIT(1);

  '''  
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


## Вывод
- Книги **Джоан Роллинг** имею самую высокую оценку среди книг, имеющих 50 и более оценок

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [8]:
query = '''
WITH
d AS
(
WITH
username AS
    (SELECT username
     FROM ratings 
     GROUP BY username
     HAVING COUNT(rating) > 50)
    
SELECT r.username,
       COUNT (r.review_id) AS count_review
FROM reviews AS r
JOIN username AS u ON r.username = u.username
GROUP BY (r.username))

SELECT ROUND(AVG(count_review))
FROM d

'''
pd.io.sql.read_sql(query, con = engine)

,round
0,24.0


## Вывод
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок  = **24**

## Рекомендации
- Новым продуктом может быть новая книга жанра фантастика современного автора или книга известного автора, которой еще нет в библиотеке
- Хорошим решеним также будет добавить аудиокниги для расширения аудитории